In [1]:
!pip install 'zarr<3'
!pip install timm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## Step 2: Import VitaminP modules

In [2]:
# Cell 2: Import VitaminP
from vitaminp import VitaminPFlex, VitaminPDual
from vitaminp.inference import WSIPredictor
from vitaminp.inference.wsi_handler import WSIHandler
from vitaminp.inference.utils import setup_logger

print("✓ VitaminP imports successful!")
print(f"  - VitaminPFlex: {VitaminPFlex}")
print(f"  - VitaminPDual: {VitaminPDual}")
print(f"  - WSIPredictor: {WSIPredictor}")

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminP imports successful!
  - VitaminPFlex: <class 'vitaminp.models.VitaminPFlex'>
  - VitaminPDual: <class 'vitaminp.models.VitaminPDual'>
  - WSIPredictor: <class 'vitaminp.inference.predictor.WSIPredictor'>


## Step 3: Initialize Model

In [3]:
import torch
# Cell 3: Initialize VitaminP Model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize your model (use the same one you trained)
model = VitaminPFlex(
    model_size='large',  # Change to your model size: 'small', 'base', 'large', 'xlarge'
    freeze_backbone=False
)

print(f"✓ Model initialized: {model.__class__.__name__}")
print(f"  Model size: large")
print(f"  Device: {device}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")

✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]
✓ Model initialized: VitaminPFlex
  Model size: large
  Device: cuda
  Total parameters: 469,380,236


## Step 4: Create WSIPredictor

In [4]:
# Cell 4: Create WSIPredictor (UPDATED)
checkpoint_path = 'checkpoints/vitamin_p_flex_large_fold2_best.pth'
output_dir = 'results/test_inference'

predictor = WSIPredictor(
    model=model,
    checkpoint_path=checkpoint_path,
    device=device,
    batch_size=None,
    patch_size=512,  # CHANGED from 1024 to 512
    overlap=64,
    target_mpp=0.3225,
    magnification=40,
    mixed_precision=True,
    num_workers=4,
)

print("✓ WSIPredictor created successfully!")
print(predictor)

2025-12-24 05:18:32 - vitaminp.inference - INFO - Loading checkpoint: checkpoints/vitamin_p_flex_large_fold2_best.pth
2025-12-24 05:18:36 - vitaminp.inference - INFO - Checkpoint loaded successfully
2025-12-24 05:18:36 - vitaminp.inference.tile_processor - INFO - GPU memory: 84.93 GB
2025-12-24 05:18:36 - vitaminp.inference.tile_processor - INFO - Estimated batch size: 25
2025-12-24 05:18:36 - vitaminp.inference - INFO - WSIPredictor initialized: device=cuda, batch_size=25, patch_size=512, overlap=64, target_mpp=0.3225, magnification=40


✓ WSIPredictor created successfully!
WSIPredictor(
  model=VitaminPFlex,
  checkpoint=vitamin_p_flex_large_fold2_best.pth,
  device=cuda,
  batch_size=25,
  patch_size=512,
  overlap=64,
  magnification=40
)


## Step 6: Run Full Inference!

In [6]:
# Cell: Full WSI Inference (Final Test)
test_wsi_path = '/rsrch9/home/plm/idso_fa1_pathology/TIER1/yasin-vitaminp/ORION-CRC/CRC21/19510_C21_US_SCAN_OR_001__153607-registered.ome.tif'
output_dir = 'results/full_inference'

wsi_properties = {
    'slide_mpp': 0.325,
    'magnification': 40
}

results = predictor.predict(
    wsi_path=test_wsi_path,
    output_dir=output_dir,
    branch='he_nuclei',  # or 'he_cell', 'mif_nuclei', 'mif_cell'
    wsi_properties=wsi_properties,
    filter_tissue=True,
    tissue_threshold=0.1,
    clean_overlaps=True,
    iou_threshold=0.5,
    save_heatmap=True,
    save_json=True,
    save_csv=True,
)

print("\n" + "="*60)
print("INFERENCE COMPLETE!")
print("="*60)
print(f"Branch: {results['branch']}")
print(f"Total detections: {results['num_detections']}")
print(f"Processing time: {results['processing_time']:.2f} seconds")
print(f"Results saved to: {output_dir}")

2025-12-24 05:18:38 - vitaminp.inference - INFO - Starting inference on WSI: 19510_C21_US_SCAN_OR_001__153607-registered.ome.tif
2025-12-24 05:18:38 - vitaminp.inference - INFO - Using branch: he_nuclei
2025-12-24 05:18:38 - vitaminp.inference - INFO - Step 1/5: Loading WSI and generating tiles...
2025-12-24 05:18:38 - vitaminp.inference - INFO - Loading WSI: 19510_C21_US_SCAN_OR_001__153607-registered.ome.tif
2025-12-24 05:18:39 - vitaminp.inference - INFO - Using provided slide_mpp: 0.325
2025-12-24 05:18:39 - vitaminp.inference - INFO - Using provided magnification: 40
2025-12-24 05:18:39 - vitaminp.inference - WARNING - Slide MPP (0.325) outside optimal range. Using requested target_mpp: 0.3225. Handle with care!
2025-12-24 05:18:39 - vitaminp.inference - INFO - Generating tiles for WSI: 90253x49683 (downsampling: 0.992)
2025-12-24 05:18:39 - vitaminp.inference - INFO - Generated 22311 tiles
2025-12-24 05:18:39 - vitaminp.inference - INFO - WSI loaded successfully. Total tiles: 223


INFERENCE COMPLETE!
Branch: he_nuclei
Total detections: 9518
Processing time: 583.77 seconds
Results saved to: results/full_inference
